In [1]:
import MDAnalysis as mda
import numpy as np

/home/simon/.local/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [2]:
path_to_data = "../level1/breaking-a-carbon-nanotube/breakable-bonds/"
u = mda.Universe(path_to_data + "cnt_deformed.data", 
                 path_to_data + "dump.lammpstrj",
                 topology_format="data",
                 format="lammpsdump",
                 atom_style='id type x y z',
                 guess_bonds=True, vdwradii={'1':1.7})

/home/simon/.local/lib/python3.11/site-packages/MDAnalysis/coordinates/LAMMPS.py:598: UserWarning: Reader has no dt information, set to 1.0 ps
  ts.data['time'] = step_num * ts.dt


In [3]:
cnt = u.select_atoms("type 1")

In [4]:
lbond_vs_frame = []
nbond_vs_frame = []
for ts in u.trajectory:
    frame = ts.frame
    all_bonds_ts = []
    for id1, id2 in cnt.atoms.bonds.indices:
        pos1 = u.atoms.positions[u.atoms.indices == id1]
        pos2 = u.atoms.positions[u.atoms.indices == id2]
        r = np.sqrt(np.sum((pos1-pos2)**2))
        if r < 1.8:
            all_bonds_ts.append(r)
    mean_length_bonds = np.mean(all_bonds_ts)
    number_of_bond = len(all_bonds_ts)/2 # divide by 2 to avoid counting twice
    lbond_vs_frame.append([frame, mean_length_bonds]) 
    nbond_vs_frame.append([frame, number_of_bond])

In [ ]:
np.savetxt("number_bond_vs_time.dat", nbond_vs_frame)
np.savetxt("length_bond_vs_time.dat", lbond_vs_frame)

In [ ]:
bond_length_distributions = []
for ts in u.trajectory:
    all_bonds_ts = []
    for id1, id2 in cnt.atoms.bonds.indices:
        pos1 = u.atoms.positions[u.atoms.indices == id1]
        pos2 = u.atoms.positions[u.atoms.indices == id2]
        r = np.sqrt(np.sum((pos1-pos2)**2))
        if r < 1.8:
            all_bonds_ts.append(r)
    if frame > 0: # ignore the first frame
        histo, r_val = np.histogram(all_bonds_ts, bins=50, range=(1.3, 1.65))
        r_val = (r_val[1:]+r_val[:-1])/2
        bond_length_distributions.append(np.vstack([r_val, histo]))
starting_distribution = np.mean(bond_length_distributions[0:20], axis=0)
end_distribution = np.mean(bond_length_distributions[200:220], axis=0)

In [ ]:
np.savetxt("starting_bond_distribution.dat", starting_distribution.T)
np.savetxt("ending_bond_distribution.dat", end_distribution.T)